# OPA Binance Stream

## Import

In [11]:
# pip install websocket-client

import websocket
import json
from binance.client import Client
import xml.etree.ElementTree as ET

from datetime import datetime
import datetime, time

import sqlite3, sqlalchemy

# Chargement des modules "Client" et "AscynClient" du package "python-binance"
from binance.client import Client, AsyncClient # dans Conda : pip install python-binance
# chargement du module pandas
import pandas as pd


In [12]:
# Chemin vers votre fichier XML
# chemin_fichier_xml = "C:/Users/psvt05771/OneDrive - SNCF/Documents/Perso/MasnewS/DataScientest/Projet OPA/SV/Binance_Key.xml"
chemin_fichier_xml = "./Binance_Key.xml"


# Parsez le fichier XML
tree = ET.parse(chemin_fichier_xml)
root = tree.getroot()


for Binance in root.findall('Binance'):
    Binance_Key = Binance.find('api_key').text
    Binance_Secret = Binance.find('api_secret').text
    

## Connexion API Binance


In [13]:

# Clé et secret pour la connexion à l'API BINANCE
api_key = Binance_Key
api_secret = Binance_Secret

In [14]:
 # récupération des clefs privés de connexion API
binance_client = Client(api_key, api_secret)

## Database

In [15]:
%load_ext sql
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = 50
%sql sqlite:///opa_database.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [16]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect

engine = create_engine('sqlite:///opa_database.db', echo= True)
conn = engine.connect()

In [17]:
%%sql
--Création de la table T_CRYPTO_STREAM
drop table if exists T_CRYPTO_STREAM ;

[]

In [18]:
%%sql
create table if not exists T_CRYPTO_STREAM
    (
          symbol                varchar(10)   not null
        , price                 float         not null
        , quantity              int           not null
        , time                  timestamp     not null
        , primary key(symbol, time)
    );

[]

## stream

In [19]:
import json
import websocket
import datetime

def ws_trades(symbol): 
    socket = f'wss://stream.binance.com:9443/ws/{symbol}@trade'
    # print(socket)
    # socket_ethusdt = f'wss://stream.binance.com:9443/ws/ethusdt@trade'
    def on_message(wsapp,message):  
        json_message = json.loads(message)
        handle_trades(json_message)
        # return test

    def on_error(wsapp,error):
        print(error)
    
    def on_close(ws):
        # df = pd.DataFrame(test,columns=['Symbol','price'])
        # df.to_sql('T_CRYPTO_STREAM',con=engine,if_exists='replace', index=False)
        print("Close connection")

    wsapp = websocket.WebSocketApp(socket, on_message=on_message, on_error=on_error)
    wsapp.run_forever()
    
def handle_trades(json_message, arr_Symb=[],arr_price=[],arr_datetime=[], arr_qty= []):
    date_time = datetime.datetime.fromtimestamp(json_message['E']/1000).strftime('%Y-%m-%d %H:%M:%S')
    # print("SYMBOL: "+json_message['s'])
    # print("PRICE: "+json_message['p'])
    # print("QTY: "+json_message['q'])
    # print("TIMESTAMP: " + str(date_time))
    # print("-----------------------")
    
    # Ajout dans un dataframe
    # data_arr = list(json_message['s'],json_message['p'],json_message['q'], date_time)
    # arr_Symb = []
    # arr_price = []
    arr_Symb.append(json_message['s'])
    arr_price.append(json_message['p'])
    arr_datetime.append(date_time)
    arr_qty.append(json_message['q'])
    # print(arr_Symb)
    # print(arr_price)
    # print(arr_datetime)

    # data_arr.append(list(zip(arr_Symb, arr_price)))

    # print(data_arr)
    # df = pd.DataFrame(data_arr,columns=['Symbol', 'price','Qty','time'])
    # df = pd.DataFrame(data_arr,columns=['Symbol','price'])
    # return data_arr
    # print(df)
    
    # print(len(arr_Symb))
    if len(arr_Symb) == 10 :
        # print('ok')
        data_arr = list(zip(arr_Symb, arr_price,arr_qty,arr_datetime))
        arr_Symb.clear()
        arr_price.clear()
        arr_datetime.clear()
        arr_qty.clear()
        df = pd.DataFrame(data_arr,columns=['Symbol','price','quantity','time'])
        df.to_sql('T_CRYPTO_STREAM',con=engine,if_exists='append', index=False)


    # ajout DB
    # df.to_sql('T_CRYPTO_STREAM',con=engine,if_exists='replace', index=False)
    # print("added data to database")

In [20]:
# Symbol : ethusdt, btcusdt, solusdt, bnbusdt
ws_trades('btcusdt')

[WinError 10054] Une connexion existante a dû être fermée par l’hôte distant


In [12]:
%%sql
SELECT *
FROM T_CRYPTO_STREAM

symbol,price,quantity,time
BTCUSDT,69452.0,7e-05,2024-05-21 21:39:43
BTCUSDT,69451.99,0.0224,2024-05-21 21:39:44
BTCUSDT,69441.41,9e-05,2024-05-21 21:39:45
BTCUSDT,69437.2,0.0133,2024-05-21 21:39:47
BTCUSDT,69437.2,0.01079,2024-05-21 21:39:48
BTCUSDT,69437.2,0.01656,2024-05-21 21:39:49
BTCUSDT,69437.2,0.00419,2024-05-21 21:39:50
BTCUSDT,69437.2,0.04101,2024-05-21 21:39:51
BTCUSDT,69437.21,0.00042,2024-05-21 21:39:53
BTCUSDT,69437.21,9e-05,2024-05-21 21:39:55


### fermeture connexion

In [21]:

# c.close
conn.close
engine.dispose

<bound method Engine.dispose of Engine(sqlite:///opa_database.db)>